In [6]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('../rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 2200
# Introduction to Algorithms

## Divide and Conquer


Today's agenda:

- Divide-and-Conquer Framework
- Euclidean TSP

We've seen a few divide-and-conquer algorithms already, so let's look at the high-level approach. For a problem $A$ and instance $\mathcal{I}_A$:

- **Base Case**: If $\mathcal{I}_A$ is small, solve directly. 

- **Inductive Step**: 
    - **Divide** $\mathcal{I}_A$ into smaller instances.
    - **Recursively solve** smaller instances.
    - **Combine** solutions

As we'll see, each algorithmic paradigm has high-level strategies to i) *prove correctness* and ii) *determine work/span*.

How do we prove the correctness of a divide-and-conquer algorithm?


Induction -- why and how?

Induction provides a natural framework for divide-and-conquer algorithms.

The **base case** of the induction requires us to prove that the algorithm works for the base case.

For the **induction step**, we use the inductive hypothesis that the solutions to the smaller instances are correct. Then, we must prove that the combine step correctly produces the desired solution. 

What about determining work/span?

We've seen that recurrences can capture the behavior of divide-and-conquer algorithms - they simply capture the cost of recursively solving smaller instances and then combining the solutions. 

The general form of the work is:

$$ W(n) = W_{\mathsf{\small divide}}(n) + \sum_{i=1}^{k}W(n_i) + W_{\mathsf{\small combine}}(n) + 1 $$

The general form for the span is:

$$ S(n) = S_{\mathsf{\small divide}}(n) + \max_{i=1}^{k}S(n_i) + S_{\mathsf{\small combine}}(n) + 1 $$



Let's look at how Merge Sort fits into this framework.


<p><span class="math display">\[\begin{array}{l}  
\mathit{mergeSort}~a =  
\\   
~~~~\texttt{if}~|a| \leq 1~\texttt{then}  
\\   
~~~~~~~~a  
\\  
~~~~\texttt{else}  
\\   
~~~~~~~~\texttt{let}  
\\  
~~~~~~~~~~~~(l,r) = \mathit{splitMid}~a  
\\   
~~~~~~~~~~~~(l',r') = (\mathit{mergeSort}~l \mid\mid{} \mathit{mergeSort}~r)  
\\  
~~~~~~~~\texttt{in}  
\\   
~~~~~~~~~~~~\mathit{merge} (l',r')  
\\  
~~~~~~~~\texttt{end}  
\end{array}\]</span></p>




For the correctness, we need to show that Merge Sort truly sorts the list. Let's perform induction:

- **Base case**: We correctly sort a singleton list.

- **Induction Step**: We assume that we can correctly sort the two halves of the list (by the inductive hypothesis). The final step is to prove that the merge step works correctly to combine two sorted lists into one sorted list (which we've shown previously). 


For the running time, recall that we characterized the work/span of Merge Sort as:

$$ W(n) = 2W(n/2) + O(n) $$
 
and

$$ S(n) = S(n/2) + O(\log n) $$
 
This fits into the framework above: the divide step takes $O(1)$ time, there are 2 concurrent recursive calls, and merging takes $O(n)$ work and $O(\log n)$ span.    

## The Euclidean Traveling Salesperson Problem

In the Euclidean Traveling Salesperson Problem (eTSP), you are given a set of $n$ 2D points. The goal is to find a "tour" (of the points with minimum cost. That is, we must construct a sequence of all the points (i.e., a sequence of 2D points) that begins and ends with the same point such that:

- every point is visited exactly once (except the starting point) 
- the sum of distances between adjacent points is minimized

This is an incredibly widespread and useful problem -- consider all the various kinds of routing problems that are solved every day. For a simple example, think of Amazon/USPS/UPS package deliveries.

Which solution is better?

<br><p> 
 ![eTSP_simple.jpg](eTSP_simple.jpg)
<br><p> 

Given an input with $n$ points, how many possible solutions are there?

## Brute-Force?

If we take a brute-force approach to this problem, what is the solution space and how can we search it?

There are $n!$ possible solutions, and we must check the cost of each by summing $n-1$ distances. This can be done with $O(n)$ work and $O(\log n)$ span. So we can solve eTSP with $O(n\cdot n!)$ work and $O(\log n)$ span. 

This is good span, but an astronomical amount of work. What if we had more points?

<br><p> 
 ![eTSP_harder.jpg](eTSP_harder.jpg)
<br><p> 

16! is about $2 x 10^{13}$, so while there are very few points the brute-force approach is not tractable!

Is the brute-force algorithm work-efficient?


## Divide-and-Conquer?

What intuition can we get about the fact that this problem is in 2D?


<br><p> 
 ![eTSP_harder_sol.jpg](eTSP_harder_sol.jpg)
<br><p> 

Since points that are "clustered" can possibly be dealt with first, how about a divide-and-conquer approach? How would that work?



We can split the input using a "cut" through the plane that separates the input points into two equal parts. Then, recursively solve eTSP for each smaller point set. 

How do we combine smaller solutions into larger ones?



We need to make sure that two tours can be combined into the best possible single tour.

<br><p> 
 ![eTSP_merge.jpg](eTSP_merge2.jpg)
<br><p> 

To do this, we can try all possible ways to merge each tour by rerouting across the cut and back and choose the least costly. This yields the following algorithm:


<p><span class="math display">\[\begin{array}{l}  
\mathit{eTSP}~(P) =  
\\  
~~~~\texttt{if}~|P|<2~\texttt{then}  
\\  
~~~~~~~~\texttt{raise}~\mathit{TooSmall}  
\\  
~~~~\texttt{else if}~|P| = 2~\texttt{then}  
\\  
~~~~~~~~\left\langle\, (P[0],P[1]),(P[1],P[0]) \,\right\rangle  
\\  
~~~~\texttt{else}  
\\  
~~~~~~~~\texttt{let}  
\\  
~~~~~~~~~~~~(P_\ell, P_r) = \mathit{split}~P~\texttt{along the longest dimension}  
\\  
~~~~~~~~~~~~(L, R) = (\mathit{eTSP}~P_\ell) \mid\mid{} (\mathit{eTSP}~P_r)  
\\  
~~~~~~~~~~~~(c,(e,e')) = \mathit{minVal}_{\mathit{first}} \left\{ (\mathit{swapCost}(e,e'),(e,e')) : e \in L, e' \in R \right\}  
\\  
~~~~~~~~\texttt{in}  
\\  
~~~~~~~~~~~~~~~~\mathit{swapEdges}~(\mathit{append}~(L,R),e,e')  
\\  
~~~~~~~~\texttt{end}  
\end{array}\]</span></p>

<p>The function $\mathit{minVal}_{\mathit{first}}$ uses the first value of the pairs to find the minimum, and returns the (first) pair with that minimum. The function $\mathit{swapEdges}(E,e,e')$ finds the edges $e$ and $e'$ and swaps the endpoints. As there are two ways to swap, it picks the cheaper one.</p>



**Correctness**: Does this algorithm compute a tour? Does this algorithm compute a minimum-cost tour?
    
We can show by induction that this algorithm always produces a tour. 

### However, the combine step does not necessarily produce a minimum cost tour!

<br>

Actually, we currently do not know of any  polynomial-work algorithm to solve this problem. In fact, the brute-force algorithm is essentially the best we can do. (We'll get to this in more detail at the end of the semester.)

What we do know how to do efficiently is to compute an *approximation* to the optimal eTSP solution. We can compute a solution that is within $(1+\epsilon)$ of optimal. The running time is polynomial in $n$ and $1/\epsilon$. 

This algorithm is actually not correct in the sense that it is not necessarily an approximation to the optimal solution. Rather, it is a *heuristic* that works well in practice.

**Work/Span**:

This algorithm has two recursive calls that each operate on $n/2$ points. To combine the solution we must check $O(n^2)$ ways too cross the cut and compute the best. This requires $O(n^2)$ work and $O(\log n)$ span. 

So we have that the work is $W(n) = 2W(n/2) + O(n^2).$ This is a root-dominated recurrence, and thus $W(n) = O(n^2)$. 

The span is $S(n) = S(n/2) + O(\log n)$. This is a balanced recurrence with $\lg n$ levels, and so $S(n) = O(\log^2 n)$.
